In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
import json
import osmnx as ox

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%pwd

'/home/cseadmin/dz/TrafficFlowModel/data_process/whc'

In [2]:
DATA_PATH = "../../data/"
TAXI_DATA_PATH = "../../data/taxi_after_proc/clean202006"
DATASET = "whc"

MIN_LAT = 22.518177550691007
MAX_LAT = 22.55855667275185
MIN_LNG = 114.04103544140904
MAX_LNG = 114.099681037508

START_DAY = 1
END_DAY = 30

DOWNSAMPLING_INTERVAL = 10 #s
TRAJ_SPLIT_INTERVAL = 600
FLOW_AGG_INTERVAL_MINUTE = 5

def contains(lat, lng):
    return lat >= MIN_LAT and lat <= MAX_LAT and lng >= MIN_LNG and lng <= MAX_LNG

In [3]:
import math

def distance(origin, destination):
    """
    Calculate the Haversine distance.

    Parameters
    ----------
    origin : tuple of float
        (lat, long)
    destination : tuple of float
        (lat, long)

    Returns
    -------
    distance_in_km : float

    Examples
    --------
    >>> origin = (48.1372, 11.5756)  # Munich
    >>> destination = (52.5186, 13.4083)  # Berlin
    >>> round(distance(origin, destination), 1)
    504.2
    """
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371  # km
    
    if lat1 == -1 or lon1 == -1 or lat2 == -1 or lon2 == -1:
        return 0

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d

In [ ]:
gps_file = open(os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "gps.csv"), "w")
trash = gps_file.write("id;x;y;time;speed\n")

timedelta_downsampling = pd.Timedelta(seconds=DOWNSAMPLING_INTERVAL)
timedelta_traj_split = pd.Timedelta(seconds=TRAJ_SPLIT_INTERVAL)

traj_counter = 0
for taxi_file in tqdm(sorted(os.listdir(TAXI_DATA_PATH))):
    df_taxi = pd.read_pickle(os.path.join(TAXI_DATA_PATH, taxi_file))
    trash=df_taxi.drop_duplicates("gps_time", inplace=True) # 这一步其实该在数据清洗的时候做: 一辆车同一时间出现在不同位置
    if len(df_taxi) < 2:
        continue
    
    # check first row's speed
    first_row = df_taxi.iloc[0]
    if first_row["speed"] == 0:
        second_row = df_taxi.iloc[1]
        dist = distance((first_row["lat"], first_row["lng"]), (second_row["lat"], second_row["lng"]))
        time_delta = (second_row["gps_time"] - first_row["gps_time"]).total_seconds() / 3600 # hour
        df_taxi.loc[df_taxi.index[0], "speed"] = dist / time_delta # first_row is a copy, must operate on df

    line_buffer = []
    # (index, lat, lon, time, speed)
    last_row = (-1, -1, -1, df_taxi.iloc[0]["gps_time"] + pd.Timedelta(seconds=-TRAJ_SPLIT_INTERVAL), -1)
    # last_time = df_taxi.iloc[0]["gps_time"] + pd.Timedelta(seconds=-TRAJ_SPLIT_INTERVAL)
    for row in df_taxi.itertuples():
        row = list(row)
        if not contains(row[1], row[2]):
            continue
        if row[3] - last_row[3] < timedelta_downsampling:  # resample: drop <30s
            continue
        # if row[4] > 60: # drop speed > 60km/h
        #     continue
        if row[4] == 0:
            dist = distance((row[1], row[2]), (last_row[1], last_row[2]))
            time_delta = (row[3] - last_row[3]).total_seconds() / 3600 # hour
            row[4] = dist / time_delta
        if row[3] - last_row[3] > timedelta_traj_split:
            if len(line_buffer) > 1:  # only store length>1 traj
                trash = gps_file.write("".join(line_buffer))
                traj_counter += 1
            line_buffer = []

        last_row = row

        line_buffer.append(f"{traj_counter};{row[2]};{row[1]};{row[3]};{row[4]}\n")

gps_file.close()


  8%|▊         | 51843/649565 [23:36<4:57:06, 33.53it/s] 

In [ ]:
# https://github.com/cyang-kth/fmm/issues/166
# https://github.com/cyang-kth/fmm/blob/master/example/osmnx_example/README.md

os.system(
    "ubodt_gen --network {} --network_id fid --source u --target v --output {} --delta 0.03 --use_omp"
    .format(
        os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "edges.shp"),
        os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "ubodt.txt")))
os.system(
    "fmm --ubodt {} --network {} --network_id fid --source u --target v --gps {} --gps_point -k 8 -r 0.003 -e 0.0005 --output {} --use_omp --output_fields id,opath,cpath > {} 2>&1"
    .format(
        os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "ubodt.txt"),
        os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "edges.shp"),
        os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "gps.csv"),
        os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "mr.txt"),
        os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "fmm.log")))